In [57]:
# custom imports
import os
import sys

# import module 
import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt 

In [58]:
# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

In [59]:
# store all stopwords
additional_stopwords = []
stopwords = nltk.corpus.stopwords.words('english') + additional_stopwords

In [60]:
# function to cleanup text and find words in it
def extract_sentences_from_text(text):

    #sentence list
    return [sentence.strip() for sentence in text.split('.') if sentence != ""]

In [61]:
def extract_vocabulary():
    curr_path = os.path.abspath("testing_vocab_match_glamour_magazine.ipynb")
    df_path = os.path.abspath(os.path.join(curr_path, "../../..", "Read_Files/fashion_vocabulary.csv"))
    df = pd.read_csv(df_path)

    vocabulary_list = [v.lower() for v in df['Specifications']]

    return vocabulary_list

In [62]:
def edit_distance(words, vocabulary):
    lis = []
    
    for extracted_word in words:
        for vocab in vocabulary:
            cnt = 0
            for i in range(max(len(vocab), len(extracted_word))):
                if i >= len(vocab) or i >= len(extracted_word):
                    cnt += 1
                elif vocab[i] != extracted_word[i]:
                    cnt += 1
            
            lis.append([vocab, extracted_word, cnt])

    df = pd.DataFrame(lis, columns=["Vocaubulary", "Extracted word", "Edit distance"])
    df.sort_values(by="Edit distance", inplace=True)

    print(df)

    all_matching_words = set()
    
    for index, row in df.iterrows():
        if row["Edit distance"] == 0:
            all_matching_words.add(row["Extracted word"])
    
    print(all_matching_words)


In [63]:
# getting page content
html_response = requests.get("https://www.glamourmagazine.co.uk/gallery/spring-summer-2022-fashion-trends")
html_text = html_response.text
soup = BeautifulSoup(html_text, "lxml")

In [ ]:
# various sources of text
para_texts = [element.text.strip() for element in soup.find_all("p")]
for p in range(len(para_texts)):
    if para_texts[p] == "":
        continue
    if para_texts[p][-1] != '.':
        para_texts[p] += '.'

header_texts = [element.text.strip() for element in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])]
for p in range(len(header_texts)):
    if header_texts[p] == "":
        continue
    if header_texts[p][-1] != '.':
        header_texts[p] += '.'

span_texts = [element.text.strip() for element in soup.find_all("span")]
for p in range(len(span_texts)):
    if span_texts[p] == "":
        continue
    if span_texts[p][-1] != '.':
        span_texts[p] += '.'

all_texts = para_texts + header_texts + span_texts
all_texts_string = ""
for v in all_texts:
    all_texts_string += v

all_texts_string = all_texts_string.lower()

print(all_texts_string)

In [ ]:
sentences = extract_sentences_from_text(text=all_texts_string)
print(len(sentences))
vocabulary = extract_vocabulary()
print(vocabulary)

In [70]:
list_of_sentences = list()
set_of_sentences = set()

for v in vocabulary:
    l = [word for word in v.split(' ')]
    if len(l) > 2:
        continue
    for sentence in sentences:
        sentence_split = [word for word in sentence.split(' ')]
        cur = -1
        for i in range(len(sentence_split)):
            if sentence_split[i].startswith(l[-1]) == True:
                cur = i
                break
        
        if cur == -1:
            continue

        if len(l) == 1 or len(sentence_split) == 1:
            set_of_sentences.add(sentence)
        else:
            j = i-1
            k = i+1

            while j >= 0 and i-j <= 3:
                if sentence_split[j].startswith(l[-2]) == True:
                    set_of_sentences.add(sentence)
                    break
                j -= 1

            while k < len(sentence_split) and k-i <= 3:
                if sentence_split[k].startswith(l[-2]) == True:
                    set_of_sentences.add(sentence)
                    break
                k += 1            

print(len(set_of_sentences))
for p in set_of_sentences:
    print(p)

1
whether it's a top with multi-straps across the shoulders and décolletage, trousers with a strapped waistband criss-crossing across the stomach – or, indeed, both – you'll have to forgo any worries of awkward tan lines next season and embrace the straps
